In [ ]:


batch_dirs = [os.path.join(INPUT_DIR, batch) for batch in os.listdir(INPUT_DIR) if not BATCHES or int(re.findall(reg_str, batch)[0]) in BATCHES]


In [47]:
import os
import pickle
import pandas as pd
import re
from tqdm.notebook import tqdm

def import_lemmatised_paragraphs(INPUT_DIR, POS, BATCHES=[], ONLY_ENGLISH_WORDS=False):
    BATCHES = [int(x) for x in BATCHES]
    reg_str = 'biggest_cities_([0-9]+)'
    
    batch_dirs = [batch for batch in os.listdir(INPUT_DIR) if not BATCHES or int(re.findall(reg_str, batch)[0]) in BATCHES]
    
    data_dict = {}
    for batch_name in tqdm(batch_dirs, desc=f"BATCHES: {BATCHES}"):
        batch_dir = os.path.join(INPUT_DIR, batch_name)
        
        for citypair in tqdm(os.listdir(batch_dir), desc="City Pair", leave=False):
            citypair_dir = os.path.join(batch_dir, citypair)
            CITY_PAIR = citypair.split('___')[1]
            
            paragraphs_count = len(pd.read_csv(f"{citypair_dir}/{CITY_PAIR}.csv"))
            data_dict[CITY_PAIR] = {'batch': batch_name, 'original_paragraphs': paragraphs_count, 'english_words': ONLY_ENGLISH_WORDS}
            
            
            
            for tag in POS:
                if ONLY_ENGLISH_WORDS:
                    file_path = f"{citypair_dir}/lemmatisation/{tag}_CLEAN.pickle"
                else:
                    file_path = f"{citypair_dir}/lemmatisation/{tag}_CLEAN.pickle"
                
                if os.path.exists(file_path):
                    with open(file_path, 'rb') as fp:
                        lemmatised_paragraphs = pickle.load(fp)

                        data_dict[CITY_PAIR][tag] = lemmatised_paragraphs
    
    # Check if all lemmatisation files were present
    missing = {k: [] for k in POS} 
    for citypair in data_dict.keys():
        for tag in POS:
            if tag not in data_dict[citypair]:
                missing[tag].append(citypair)
    
    for k in missing:
        if len(missing[k]):
            print(f"The following city pairs have missing '{k}' files: \n--> {missing[k]}\n")
            
    print(f'\n Getting lemmatised paragraphs for {len(data_dict.keys())} city pairs...')
    
    return data_dict

In [50]:
INPUT_DIR = "../../../../../data/clean/city_pair_paragraphs/"
BATCHES = [10]
POS = ["VERB"]
# LEMMATISATION_TYPE = 'quick', 'accurate'
ONLY_ENGLISH_WORDS = True
OVERWRITE = False


a = import_lemmatised_paragraphs(INPUT_DIR, POS, BATCHES, ONLY_ENGLISH_WORDS)

BATCHES: [10]:   0%|          | 0/1 [00:00<?, ?it/s]

City Pair:   0%|          | 0/35 [00:00<?, ?it/s]

The following city pairs have missing 'VERB' files: 
--> ['athens_birmingham', 'athens_lisbon', 'athens_rome', 'barcelona_athens', 'barcelona_birmingham', 'barcelona_lisbon', 'barcelona_rome', 'berlin_athens', 'berlin_barcelona', 'berlin_birmingham', 'berlin_lisbon', 'berlin_rome', 'birmingham_lisbon', 'london_athens', 'london_barcelona', 'london_birmingham', 'london_lisbon', 'london_rome', 'madrid_athens', 'madrid_barcelona', 'madrid_birmingham', 'madrid_lisbon', 'madrid_rome', 'milan_athens', 'milan_barcelona', 'milan_birmingham', 'milan_lisbon', 'milan_rome', 'paris_athens', 'paris_barcelona', 'paris_birmingham', 'paris_lisbon', 'paris_rome', 'rome_birmingham', 'rome_lisbon']


 Getting lemmatised paragraphs for 35 city pairs...


In [43]:
a['athens_bucharest']

{'batch': 'biggest_cities_20',
 'original_paragraphs': 150,
 'english_words': True}

In [7]:
len(pd.read_csv('../../../../../data/clean/city_pair_paragraphs/biggest_cities_5/cities___paris_london___/paris_london.csv'))

21051

In [ ]:
import os
import pickle

word_list_location = "../../../../data/enwiki_city_pairs_lemmatised/lemmatised_paragraphs/"
city_pair_wordlists = []
city_pairs = []
for file in os.scandir(word_list_location):
    with open(file.path, 'rb') as fp:
        city_pair_wordlists.append(pickle.load(fp))
        city_pairs.append(file.name.split('__')[1])